In [2]:
with open("./input.txt", "r",encoding="utf-8") as file:
    data = file.read()
    # print(data)
import torch

voc=sorted(list(set(data)))
# print(len(data))
# print(list(set(data)))
# print(set(data))
# print(voc)
chtoi={c:i for i,c in enumerate(voc)}
itoch={i:c for i,c in enumerate(voc)}
encoding=lambda s: [chtoi[c] for c in s]
decoding=lambda s: "".join([itoch[c] for c in s])


# print(encoding("hello world"))
# print(decoding(encoding("hello world")))

datatset=torch.tensor(encoding(data),dtype=torch.long)
# print(datatset[:100])
n=int(0.9*len(datatset))
trainset=datatset[:n]
testset=datatset[n:]
block_size=8
x_train=datatset[:block_size+1]
y_train=datatset[1:block_size+1]
# print(x_train)
for i in range(block_size):
    context=x_train[:i+1]
    target=y_train[i]
    # print(f"context: {context}, target: {target}")
torch.manual_seed(42)
batch_size=4
block_size=8
def getbatch(split):
    data=trainset if split=="train" else testset
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack(([data[i:i+block_size] for i in ix]))
    y=torch.stack(([data[i+1:i+block_size+1] for i in ix]))
    # print(ix)
    return x,y
xb,yb=getbatch("train")
print(xb)
print(yb)

for b in range(batch_size):
    for a in range(block_size):
        context=xb[b,:a+1]
        target=yb[b,a]
        print(f"context: {context}, target: {target}")
    


/mnt/c9542373-6611-4b24-a6c7-2979563b4e53/gptdev/gptenv/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
context: tensor([57]), target: 1
context: tensor([57,  1]), target: 46
context: tensor([57,  1, 46]), target: 47
context: tensor([57,  1, 46, 47]), target: 57
context: tensor([57,  1, 46, 47, 57]), target: 1
context: tensor([57,  1, 46, 47, 57,  1]), target: 50
context: tensor([57,  1, 46, 47, 57,  1, 50]), target: 53
context: tensor([57,  1, 46, 47, 57,  1, 50, 53]), target: 60
context: tensor([1]), target: 58
context: tensor([ 1, 58]), target: 46
context: tensor([ 1, 58, 46]), target: 43
context: tensor([ 1, 58, 46, 43]), target: 56
context: tensor([ 1, 58, 46, 43, 56]), target: 43
context: tensor([ 1, 58, 46, 43, 56, 43]), target: 1
context: tensor([ 

In [3]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)


In [4]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self, xb,yb=None):
        logits=self.token_embedding_table(xb)
      
        if yb is not None:
            B,C,T=logits.shape
            logits=logits.view(B*C,T)
            yb=yb.view(B*C)

            loss=F.cross_entropy(logits,yb)
        else:
            loss=None
        return logits,loss

    def generate(self, idx, max_new_tokens): 
        # idx is (B, T) array of indices in the current context 
        for _ in range(max_new_tokens):  
            # get the predictions 
            logits, loss = self(idx)  

            # focus only on the last time step 
            # print(logits)
            logits = logits[:, -1, :]  # becomes (B, C)  
            # print(logits)

            # apply softmax to get probabilities  
            probs = F.softmax(logits, dim=-1)  # (B, C)  

            # sample from the distribution  
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)  

            # append sampled index to the running sequence  
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1) 
            # print(idx) 

        return idx

m=BigramLanguageModel(len(voc))
logit,loss=m(xb,yb)
# print(logit.shape)
# print(loss)
        
# print(len(voc))

print(decoding(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


uoiaF$z
M?kI;h
DbuMG,H3LYNmrDxKgTpvAKOF-jU.hc;fBMTGa-IS
g3lEb&ZQ,l;:m;lpcNN
KpVEYRIIM,'hCRbMAcWTkrnH


In [5]:
m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)

tensor([[ 0, 16, 23, 34, 39, 51,  4, 23,  9, 38, 52, 49, 21, 25, 39, 55, 52, 58,
         48, 46,  7, 13, 35,  5, 14, 22, 62, 62, 45, 46, 48, 38, 32, 14, 15, 52,
         15, 33, 12, 55, 14, 29, 40, 61, 63, 15, 48, 46, 30, 46, 31, 24, 63, 22,
         44, 19, 58, 57, 59, 58, 22, 14, 63,  7, 48,  4,  6, 56,  6,  5, 49,  3,
         28, 32, 59, 38, 51,  9, 58, 28, 40, 59,  1, 26, 61, 20,  7, 21, 28, 43,
         39, 55, 59, 22, 34, 56, 23,  5, 62,  1, 44]])

In [6]:
len("""gBXCYUEBUkTrhlwhYvCHU3jVQWIOVFHSbBtlnSfFoHZK.nHR&AcjvfzzJ.K,HCYUui-xd
L;Wu3VnSgv.$afs
BqpdHVLgwcOwN """)

100

In [7]:
optimizer=torch.optim.AdamW(m.parameters(),lr=0.001)


In [8]:
batch_size = 32

for steps in range(10000):
    xb, yb = getbatch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if steps % 100 == 0:
        print(f"Step {steps+1}, Loss: {loss.item():.4f}")


Step 1, Loss: 4.6977
Step 101, Loss: 4.5542
Step 201, Loss: 4.4854
Step 301, Loss: 4.3880
Step 401, Loss: 4.2180
Step 501, Loss: 4.2192
Step 601, Loss: 4.0598
Step 701, Loss: 3.9743
Step 801, Loss: 3.9993
Step 901, Loss: 3.9300
Step 1001, Loss: 3.7942
Step 1101, Loss: 3.8274
Step 1201, Loss: 3.5697
Step 1301, Loss: 3.4974
Step 1401, Loss: 3.5198
Step 1501, Loss: 3.3823
Step 1601, Loss: 3.4163
Step 1701, Loss: 3.3589
Step 1801, Loss: 3.3072
Step 1901, Loss: 3.2026
Step 2001, Loss: 3.1628
Step 2101, Loss: 3.0630
Step 2201, Loss: 3.1235
Step 2301, Loss: 2.9732
Step 2401, Loss: 3.0888
Step 2501, Loss: 2.9927
Step 2601, Loss: 2.8689
Step 2701, Loss: 2.9053
Step 2801, Loss: 2.9279
Step 2901, Loss: 2.8635
Step 3001, Loss: 2.8600
Step 3101, Loss: 2.9175
Step 3201, Loss: 2.7034
Step 3301, Loss: 2.7863
Step 3401, Loss: 2.7986
Step 3501, Loss: 2.6293
Step 3601, Loss: 2.7755
Step 3701, Loss: 2.7048
Step 3801, Loss: 2.6091
Step 3901, Loss: 2.5815
Step 4001, Loss: 2.6208
Step 4101, Loss: 2.6359
Step

In [9]:
print(decoding(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


RY:
Yors t p kere isd, m INGea earusher Tistced.
Wharologo!


Whe hildy por byof?
GUFarastaldXIndesw


In [10]:
B,T,C=4,8,2
x=torch.randn(B,T,C)
xbow=torch.zeros(B,T,C)
print(x)
for b in range(B):
    for t in range(T):
        xprev=x[b,:t+1]
        xbow[b,t]=torch.mean(xprev,dim=0)

tensor([[[ 1.1583,  1.0940],
         [-0.1402,  0.1227],
         [-1.7025,  0.6909],
         [-0.5811,  1.5571],
         [-0.0903, -0.4272],
         [ 1.1420,  1.8703],
         [-0.5932, -1.6216],
         [ 0.0837,  0.4461]],

        [[-0.7926, -1.6044],
         [ 0.3674, -0.3286],
         [-1.3446,  0.2414],
         [-1.3129,  0.1608],
         [ 1.4516, -2.6770],
         [ 1.0325,  0.0754],
         [-1.2991, -0.3834],
         [-0.0684, -3.1754]],

        [[ 0.2173,  1.4930],
         [-2.7524,  1.2803],
         [-0.2602, -1.3573],
         [ 0.1989,  1.4771],
         [-0.8706, -1.1885],
         [ 0.9991, -0.3465],
         [-0.6703,  0.1645],
         [ 0.6289, -0.3070]],

        [[ 1.2607,  0.2722],
         [ 0.4268, -0.3683],
         [-0.9086,  1.8521],
         [ 2.4499, -1.9422],
         [ 0.4257,  0.0806],
         [ 0.5172,  0.9361],
         [-0.0533,  0.8518],
         [ 0.7028,  0.8040]]])


In [11]:
x[0],xbow[0]

(tensor([[ 1.1583,  1.0940],
         [-0.1402,  0.1227],
         [-1.7025,  0.6909],
         [-0.5811,  1.5571],
         [-0.0903, -0.4272],
         [ 1.1420,  1.8703],
         [-0.5932, -1.6216],
         [ 0.0837,  0.4461]]),
 tensor([[ 1.1583,  1.0940],
         [ 0.5091,  0.6083],
         [-0.2281,  0.6359],
         [-0.3164,  0.8662],
         [-0.2712,  0.6075],
         [-0.0356,  0.8180],
         [-0.1153,  0.4695],
         [-0.0904,  0.4665]]))

In [48]:
torch.manual_seed(42)
a=torch.tril(torch.ones(3,3,dtype=torch.float))
b=torch.randint(0,10,(3,2),dtype=torch.float)
c=a@b

In [49]:
a,b

(tensor([[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]]),
 tensor([[2., 7.],
         [6., 4.],
         [6., 5.]]))

In [50]:
a=a/torch.sum(a,dim=1,keepdim=True)
# b=torch.sum(a,dim=1)

a

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [52]:
v=torch.tril(torch.ones(T,T))
v=v/torch.sum(v,dim=1,keepdim=True)
v

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [53]:
# s=v/torch.sum(v,dim=1)
# s


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])